<a href="https://colab.research.google.com/github/fangyiyu/Fangyi_Yu_Queens/blob/master/Mnist_VGG_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, LSTM
from keras.utils import np_utils

In [2]:
!wget -O comp551w18-modified-mnist.zip https://www.dropbox.com/sh/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a?dl=0

--2020-09-28 12:56:27--  https://www.dropbox.com/sh/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.1, 2620:100:6023:1::a27d:4301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a [following]
--2020-09-28 12:56:27--  https://www.dropbox.com/sh/raw/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4e3eec2e9e8bd4016dfe17116c.dl.dropboxusercontent.com/zip_download_get/AjpbxfQYUlprT9g_F6oPjnkkJtskD8BPBkxlvQyHdZibGXKwo_Ai070__OuXqMZ9F658Oa541wZsEiA8Aw0hhBTshqub8PBnkX9hN6dCBGT4Jg [following]
--2020-09-28 12:56:28--  https://uc4e3eec2e9e8bd4016dfe17116c.dl.dropboxusercontent.com/zip_download_get/AjpbxfQYUlprT9g_F6oPjnkkJtskD8BPBkxlvQyHdZibGXKwo_Ai070__OuXqMZ9F658Oa541wZsEiA8A

In [4]:
!unzip comp551w18-modified-mnist.zip

Archive:  comp551w18-modified-mnist.zip
  inflating: test_x.csv              
  inflating: train_y.csv             
  inflating: train_x.csv             


In [1]:
!ls

comp551w18-modified-mnist.zip  sample_data  train_x.csv
instruction.txt		       test_x.csv   train_y.csv


# 1. Data preprocessing

In [2]:
# Load data
import numpy as np
x_train = np.loadtxt("train_x.csv", delimiter=",")# load from text
y_train = np.loadtxt("train_y.csv", delimiter=",")
x_test = np.loadtxt("test_x.csv", delimiter=",")

In [3]:
x_train=x_train.reshape(x_train.shape[0],64,64,1)
x_test=x_test.reshape(x_test.shape[0],64,64,1)

In [4]:
# Scale x
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 64, 64, 1)
50000 train samples
10000 test samples


In [ ]:
# Transform y
n_classes = 10
y_train_d = np_utils.to_categorical(y_train_c, n_classes)
y_test_d = np_utils.to_categorical(y_test_c, n_classes)
print("Shape after one-hot encoding: ", y_train_d.shape)

In [5]:
# Split dataset into train and test
from sklearn.model_selection import train_test_split
x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(x_train, y_train, test_size=0.2, random_state=7)

In [13]:
(x_train_c.shape, x_test_c.shape,y_train_d.shape,y_test_d.shape)

((40000, 64, 64, 1), (10000, 64, 64, 1), (40000, 10), (10000, 10))

In [16]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20, 
    width_shift_range=0.1,
    shear_range=0.2,
    height_shift_range=0.1,
    zoom_range=0.2
    )

datagen.fit(x_train_c)

In [ ]:
input_shape = (64,64,1)

## 2. CNN_VGG

In [17]:
# Build model
def VGG_modified(input_shape, opt = optimizers.Adam(amsgrad=True)):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation='relu', input_shape=input_shape,
                            data_format="channels_last"))
    model.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    #sgd=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [21]:
model = VGG_modified((64,64,1))
history = model.fit_generator(datagen.flow(x_train_c, y_train_d, batch_size=64),
                              validation_data=(x_test_c, y_test_d),
                              epochs = 10)

Epoch 1/10
625/625 [==============================] - 59s 94ms/step - loss: 2.3174 - accuracy: 0.1516 - val_loss: 2.2898 - val_accuracy: 0.1410
Epoch 2/10
625/625 [==============================] - 59s 94ms/step - loss: 2.1987 - accuracy: 0.1781 - val_loss: 2.1878 - val_accuracy: 0.1821
Epoch 3/10
625/625 [==============================] - 58s 93ms/step - loss: 2.1531 - accuracy: 0.1954 - val_loss: 2.1082 - val_accuracy: 0.2039
Epoch 4/10
625/625 [==============================] - 58s 93ms/step - loss: 2.0911 - accuracy: 0.2197 - val_loss: 2.2870 - val_accuracy: 0.1558
Epoch 5/10
625/625 [==============================] - 58s 93ms/step - loss: 1.9500 - accuracy: 0.2776 - val_loss: 2.0007 - val_accuracy: 0.2690
Epoch 6/10
625/625 [==============================] - 58s 93ms/step - loss: 1.7536 - accuracy: 0.3377 - val_loss: 3.4980 - val_accuracy: 0.1172
Epoch 7/10
625/625 [==============================] - 58s 93ms/step - loss: 1.5459 - accuracy: 0.4137 - val_loss: 2.3700 - val_accuracy:

# 3. RNN_LSTM

In [23]:
# Hyper parameters
batch_size = 64
nb_epoch = 10
# Parameters for MNIST dataset
img_rows, img_cols = 64, 64
nb_classes = 10
# Parameters for LSTM network
nb_lstm_outputs = 30
nb_time_steps = img_rows
dim_input_vector = img_cols
input_shape = (64, 64)

x_train_r=x_train_c.reshape(x_train_c.shape[0],64,64)
x_test_r=x_test_c.reshape(x_test_c.shape[0],64,64)

model_LSTM = Sequential()
model_LSTM.add(LSTM(nb_lstm_outputs, input_shape=input_shape))
model_LSTM.add(Dense(10, activation='softmax'))
model_LSTM.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model_LSTM.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30)                11400     
_________________________________________________________________
dense_23 (Dense)             (None, 10)                310       
Total params: 11,710
Trainable params: 11,710
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Train the model
history = model_LSTM.fit(x_train_r, 
                    y_train_d, 
                    epochs=nb_epoch, 
                    batch_size=batch_size, 
                    shuffle=True,
                    validation_data=(x_test_r, y_test_d),
                    verbose=1)

Epoch 1/10
625/625 [==============================] - 11s 18ms/step - loss: 2.3056 - accuracy: 0.1042 - val_loss: 2.3031 - val_accuracy: 0.1127
Epoch 2/10
625/625 [==============================] - 10s 17ms/step - loss: 2.2989 - accuracy: 0.1189 - val_loss: 2.2930 - val_accuracy: 0.1293
Epoch 3/10
625/625 [==============================] - 11s 17ms/step - loss: 2.2692 - accuracy: 0.1483 - val_loss: 2.2646 - val_accuracy: 0.1477
Epoch 4/10
625/625 [==============================] - 10s 17ms/step - loss: 2.2397 - accuracy: 0.1660 - val_loss: 2.2402 - val_accuracy: 0.1632
Epoch 5/10
625/625 [==============================] - 10s 17ms/step - loss: 2.2188 - accuracy: 0.1752 - val_loss: 2.2299 - val_accuracy: 0.1661
Epoch 6/10
625/625 [==============================] - 10s 17ms/step - loss: 2.2058 - accuracy: 0.1831 - val_loss: 2.2325 - val_accuracy: 0.1641
Epoch 7/10
625/625 [==============================] - 10s 17ms/step - loss: 2.1931 - accuracy: 0.1864 - val_loss: 2.2075 - val_accuracy: